## Power system modeling ##


<!-- The main motivation behind this notebook is to give an introduction to how the power system is modeled in TOPS. Further, gaining experiance with building your own models can greatly contribute to understanding how power system elements interacts with the power system. This notebook first presents the modelling approach utilized in TOPS. Then, a recipe for building a your own model is presented. -->

In this notebook, we will explore power system modeling framwork used in TOPS. We will present the core modeling approach, a simulation flowchart, and guide you through creating a dynamic model using a third-order generator as an example. By the end, you will hopefully gain some insight in how to model power system elements in in TOPS and be able to build and simulate your own models.

Written by : Lars Andreas Hermansen

Date : 03.03.2025

Last update : 04.03.2025

Still a work in progress, if there is anything missing or you have some general feedback please contact Sjur Føyen at foyen.sjur@ntnu.no 


### 🖥️ Core modeling approach
When the aim is on analyze electro-mechanical and control dynamics in power systems, it is common to represent the ac variables (such as voltage and current) as phasors. This method is often reffered to RMS-models or average models. RMS-models holds an advantage over their counterpart electromagnetic transient (EMT) models in being more computational efficient whilst still maintaining sufficient accuracy. EMT models are used when analyzing much faster dynamics at smaller timescale. The two images below shows the difference between EMT and RMS-models on how a short-circuit response might look like.


<div style="display: flex; justify-content: space-between; align-items: center;">
    <figure style="width: 45%; text-align: center;">
        <img src="Figures/A-PowerSysModeling/emt_ex.png" alt="Electromagnetic Transient (EMT) Model" style="max-width: 100%;">
        <figcaption>Electromagnetic Transient (EMT) Model</figcaption>
    </figure>
    <figure style="width: 45%; text-align: center;">
        <img src="Figures/A-PowerSysModeling/rms_ex.png" alt="RMS Model" style="max-width: 100%;">
        <figcaption>RMS Model</figcaption>
    </figure>
</div>



The power system in RMS-models is represented as a set of differential-algebraic equations (DAEs) written in the form:

$$ \dot{x} = f(x, v) $$

$$ 0 = g(x, v) $$

Differential equations $f$ describe the dynamic behavior of elements such as generators and controllers. The algebraic equations $g$ represent the relationship between node voltages $v$ and current injections in the power system. (In literature, $v$ is more commonly denoted as $y$). We will come back to this relationship later.

The main problem that needs to be solved by the simulator is integrating the differential equations whilst at the same time ensuring that the algebraic equations remain satisfied. In order to achieve this, the set of DAEs must be converted to a set of ordinary differential equations (ODEs):

The set of algebraic equations is linear and can be written as:
$$ Y \cdot V = I_{inj(x)} $$

where $Y$ is the admittance matrix of the system, $I_{inj(x)}$ is the vector of the bus current injections, and $V$ is the bus voltages. The vector containing the algebraic variables $V$ can be solved by rewriting the equation above:

$$ v = V = Y^{-1} \cdot I_{inj(x)} = h(x) $$

In practice, the voltage vector $V$ is **not** solved by inverting the admittance matrix, but rather by using an algorithmic solver. The set of differential-algebraic equations can now be written as:

$$ \dot{x} = f(x, h(x)) = f(x) $$

This system of ODEs can now be solved by using a suitable numerical integration method, for example the Euler or Modified-Euler method.





<!-- ### Simulation flow chart

The figure below shows a flow chart of the simulation process. During this notebook, we will explore how the different functionalities and equations are described in order to simulate a dynamic power system. -->

### ⚡ Power System Simulation Flowchart

The flowchart below shows the simulation process of a power system RMS-model such as TOPS. The simulation starts with an initialized power system model, including the load flow solution $\mathbf{v_0}$ and initial states $\mathbf{x_0}$. Once initiated, the simulation iterates over discrete time steps. The model calculates current injections $I_{\text{inj}}$, determines node voltages $\mathbf{V}$, and updates system states by solving the differential equations.

**Important to note**; the current injections are calculated based on the dynamic states $\mathbf{x}$ and are **independent** of the sysytem voltages. This is important to avoid alebraic loops during calculations. We will come back to how we adress this in the next section.

The **results** from the simulations consist of updated values for node voltages, and dynamic states which allows us to analyze the power system elements transient behavior and system dynamics.



<div style="text-align: center;">
    <img src="Figures/A-PowerSysModeling/flow_chart_simulation.png" alt="Flow chart of simulation" style="max-width: 70%;">
    <figcaption>Flow chart of simulation</figcaption>
</div>

<!-- <img src="Figures/flow_chart_simulation.png" width="800"> -->


## 🔧 Making your own model

Now that you have gotten an introduction to how the power system simulator works, let build our own dynamic model! The third-order generator model from the lectures will be used as an example.

In [1]:
# %pip install git+https://github.com/hallvar-h/tops

## These import methods might need to be changed when pushed to the main repository ##

# Solver and dynamic models
from tops import dynamic as dps
from tops import solvers as dps_sol
from tops.dyn_models.utils import DAEModel
import numpy as np

### 💡 Model instructions

For a model to be included in TOPS simulations, there are som key functions called "**model instructions**" that need to be correctly impelmented. These are strongly related to the simulating technique showed in the flow chart, you are going to reckognize some of the names. First, we initiate the model and defining where our model is connected to the system:



In [2]:
class gen3(DAEModel):

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.bus_idx = np.array(np.zeros(self.n_units), dtype=[(key, int) for key in self.bus_ref_spec().keys()])
        self.bus_idx_red = np.array(np.zeros(self.n_units), dtype=[(key, int) for key in self.bus_ref_spec().keys()])

    def bus_ref_spec(self):
        return {'terminal': self.par['bus']}
    
    # This is a standard initializing method in TOPS, I recommend that you just copy this method

Since our model is a power source, it needs to be included in the load flow solution. This is done in TOPS by defining a function called "**load_flow_pv**":

In [3]:
# The power is given in MW and the voltage in p.u.
def load_flow_pv(self):
    return self.bus_idx['terminal'], -self.par['P'], self.par['V']

Now that the model has been included in the power flow solution, we can use this information to calculate the initial states of our model. This is done by defining a function called "**init_from_load_flow**". The calculations are similar to what you have been doing in the course assignments. I have added the phasor diagram below the help keep track of what calculations are being done.

In [4]:
def init_from_load_flow(self, x_0, v_0, S):
    X_0 = self.local_view(x_0)
    p = self.par

    # From the power flow solution, define the per unit apperant power
    s_pu = S/p['S_n']

    # Define the generator terminal voltage and current
    v_g = v_0[self.bus_idx['terminal']]
    I_g = np.conj(s_pu/v_g)

    # print(f"Angle of {p["name"]}  I_g: ", np.angle(I_g, deg=True))

    # From the phasor diagram, E_Q can be calculated as:
    e_q_tmp = v_g + 1j * p['X_q'] * I_g

    # This allows us to find the power angle \delta
    # angle = delta
    angle = np.angle(e_q_tmp)

    # Here, we are applying a helpful trick. As the q-axis is aligned with E_Q,
    # we can use the power angle delta to rotate the dq axis to align with the real and imaginary axis
    # Meaning, by rotating the voltage and current phasors, we can decompose them into the dq frame by using the real and imaginary part

    # = e^(-j * delta)
    expdelta = np.exp(-1j * angle)

    # Now we rotate the voltage and current with the power angle delta
    # We are in practice subtracting the power angle from the angle of the voltage and current

    idq = I_g * expdelta
    vdq = v_g * expdelta

    # Now we can decompose the voltage and current into the qd frame by using the real and imaginary part
    # Note! They are no longer phasors, but decomposed values expressed in the qd frame
    id = idq.imag
    iq = idq.real
    v_g_q = vdq.real

    # Now we can calculate the initial values for the generator model
    # Take your time looking at the phasor diagram and see if you can follow the calculations
    eqt_0 = v_g_q - p['X_d_t'] * id
    e_q_0 = v_g_q - p['X_d'] * id
    
    # Set the initial values of inputs needed from other models (we will come back to these later)
    self._input_values['P_m'] = s_pu.real
    self._input_values['E_f'] = e_q_0

    # Set the initial values of our three states
    X_0['speed'][:] = 0
    X_0['angle'][:] = angle
    X_0['e_q_t'][:] = eqt_0

<div style="text-align: center;">
    <img src="Figures/A-PowerSysModeling/phasor_diagram_ts.png" alt="Phasor diagram steady-state" style="max-width: 90%;">
    <figcaption>Phasor diagram steady-state</figcaption>
</div>

Now that we have established the initial states of the model, lets take a look at the dynamics and consequently the differential equations that define the transient behavior of our model. We start by taking a look at something you are familiar with, the block diagram:


<div style="text-align: center;">
    <img src="Figures/A-PowerSysModeling/3rd_order_slide.png" alt="Third-Order Generator Model" style="max-width: 70%;">
    <figcaption>Third-Order Generator Model</figcaption>
</div>

From this model diagram, we can find our three states :

$$ x = \begin{bmatrix}
\Delta~E^{'}_{q}\\
\Delta~\delta\\
\Delta~\omega\\

\end{bmatrix} $$


Our goal is now to write differential equations describing the dynamic behavior of these states. Lets start with $\Delta~\omega$, from the block diagram we can write:

$$(\Delta~\tau_{m} - \Delta~\tau_{e} - D \cdot \Delta~\omega) \cdot \frac{1}{2Hs} = \Delta~\omega  $$

We are looking the differential equation $ \Delta~\dot{\omega} $ = ... So lets rearange the equation and notice the laplace operator $\Delta~x \cdot s = \Delta~\dot{x}$:


$$ \Delta\dot{\omega} = (1/2H) \cdot (\Delta~\tau_{m} - \Delta~\tau_{e} - D \cdot \Delta~\omega)  $$



Applying the same method to the rest of the model and we have <span style="color:lightblue">three</span> differential equations describing the dynamic behaviour of our <span style="color:lightblue">three</span> states:


$$ \dot{x} = f(x) $$

$$ \Delta~\dot{x} = \begin{bmatrix}
\Delta~\dot{E^{'}_{q}}\\
\Delta~\dot{\delta}\\
\Delta~\dot{\omega}\\
\end{bmatrix} = \begin{bmatrix}
(1/T'_{do}) \cdot (\Delta~E_{f} - E^{'}_{q} - (X_{d} - X^{'}_{d}) \cdot i_{d})\\
\Delta~\omega \cdot 2\pi f_{0}\\
(1/2H) \cdot (\Delta~\tau_{m} - \Delta~\tau_{e} - D \cdot \Delta~\omega)
\end{bmatrix} $$




The diffenerential equations are defined in TOPS via the "**state_derivatives**" function together with the "**state_list**" :

In [5]:
def state_list(self):
    return ['speed', 'angle', 'e_q_t']

def state_derivatives(self, dx, x, v):
    dX = self.local_view(dx)
    X = self.local_view(x)
    p = self.par

    p_e = self.p_e(x, v)

    dX['speed'][:] = (1 / (2 * p["H"])) * (self.P_m(x,v)/(1+X["speed"]) - p_e/(1+X["speed"]) - p['D']*X['speed'])
    dX['angle'][:] = X['speed'] * 2 * np.pi * self.sys_par['f_n']
    dX['e_q_t'][:] = (1 /p['T_d0_t']) * (self.E_f(x, v) - X['e_q_t'] + self.i_d(x, v) * (p['X_d'] - p['X_d_t']))

Notice that our model is dependent on inputs not defined within the class itself. These are inputs from external models such as the automatic voltage regulator (AVR) and turbine governor (GOV). To fascilitate this, we define an input list as shown:

In [6]:
def input_list(self):
    return ['E_f', 'v_aux', 'v_pss', "P_m"]

def int_par_list(self):
    return ['f']

Next, we need to add the generator admaittance to the diagonal element of the system admittance matrix. This is done by defining the instruction function "**dyn_const_adm**". For the third-order model, the generator is modeled as a transient d-axis reactance behind a q-axis transient induced voltage.

In [7]:
def dyn_const_adm(self):
    idx_bus = self.bus_idx['terminal']
    bus_v_n = self.sys_par['bus_v_n'][idx_bus]
    z_n = bus_v_n ** 2 / self.sys_par['s_n']

    impedance_pu_gen = 1j * self.par['X_d_t']
    impedance = impedance_pu_gen * self.par['V_n'] ** 2 / self.par['S_n'] / z_n

    Y = 1 / impedance
    return Y, (idx_bus,)*2

Now, lets define an important function ; "**current_injections**" . As shown above, TOPS uses current injections to calculate the node voltages of the system. Every power source of the system must therefore contain a current injection function in order to supply (or consume) power. Further, the calculation of the current injection cannot include the node voltage, that would cause an algebraic loop. We must therefore transform our model from a voltage source behind a reactance, to a current source in parrallel with the same reactance. This is known as a Norton-transform, you may reckognize this from electric circuit analysis courses. 

<div style="text-align: center;">
    <img src="Figures/A-PowerSysModeling/current_injections.png" alt="Norton transformation - current injection" style="max-width: 80%;">
    <figcaption>Norton transformation - current injection</figcaption>
</div>

In [8]:
def current_injections(self, x, v):
    p = self.par
    X = self.local_view(x)

    i_inj = (X["e_q_t"]*np.exp(1j*X["angle"]))/(1j * p["X_d_t"]) 

    I_n = p['S_n'] / (np.sqrt(3) * p['V_n'])

    i_n = self.sys_par['s_n']/(np.sqrt(3) * self.sys_par['bus_v_n'])

    # System p.u. base
    I_inj = i_inj*I_n/i_n[self.bus_idx_red['terminal']]

    return self.bus_idx_red['terminal'], I_inj

Lastly, it is usefull to define some utility functions to make the functions above easier to read and maintain.

In [9]:

# Utility functions
def v_t(self, x, v):
    return v[self.bus_idx_red['terminal']]

def v_t_abs(self, x, v):
    return np.abs(v[self.bus_idx_red['terminal']])

def v_setp(self, x, v):
    return self.par['V']

def e_q_t(self, x, v):
    return self.local_view(x)['e_q_t']

def angle(self, x, v):
    return self.local_view(x)['angle']

def speed(self, x, v):
    return self.local_view(x)['speed']

def i(self, x, v):
    X = self.local_view(x)
    return (self.e_q_t(x,v)*np.exp(1j*X["angle"]) - self.v_t(x, v)) / (1j*self.par["X_d_t"])

def idq(self, x, v):
    X = self.local_view(x)
    return self.i(x,v)*np.exp(-1j*X["angle"])

def i_d(self, x, v):
    return self.idq(x, v).imag

def i_q(self, x, v):
    return self.idq(x, v).real

def s_e(self, x, v):
    return self.v_t(x, v)*np.conj(self.i(x, v))

def p_e(self, x, v):
    return self.s_e(x, v).real

def q_e(self, x, v):
    return self.s_e(x, v).imag

def p_m(self, x, v):
    return self.P_m(x,v)

### References

Machowski, Jan, et al. "Power system dynamics." John Wiley & Sons, 3rd edition, 2020.